# Loading Training Data

## Notebook Test

The initial step of this project involves fetching the training data and loading it into a PostgreSQL database. The configuration for this database, when tested locally, is provided in the `docker-compose.yml` file. In the [proof of concept notebook](./01_proof_of_concept.ipynb), we demonstrated how to retrieve data from the OpenAQ API. This notebook will focus on testing the database configuration and uploading the fetched data.


In [1]:
import psycopg
import pandas as pd

To avoid making repeated calls to the OpenAQ API, we stored the results in a CSV file in [notebook 1](./01_proof_of_concept.ipynb). In this notebook, we will load that data and upload only the subset of sensors relevant to this project's objectives.

In [2]:
df = pd.read_csv("./data/sensor_data.csv")

pivot_df = df.pivot(index='datetime', columns='sensor_id', values='value').reset_index()
pivot_df = pivot_df.rename_axis(None, axis=1)
pivot_df.columns = [f'sid_{col}' if isinstance(col, int) else col for col in pivot_df.columns[:]]

pivot_df

,datetime,sid_20466,sid_20468,sid_20476,sid_34834,sid_34835,sid_34836,sid_34841,sid_34845,sid_34997,...,sid_35678,sid_35766,sid_35843,sid_36009,sid_36047,sid_36064,sid_36066,sid_36092,sid_5079414,sid_5079565
0,2022-01-01T00:00:00+00:00,9.9,24.0,11.0,24.0,25.0,0.0045,20.0,0.01600,11.0,...,32.0,0.015,41.0,18.0,14.0,19.0,41.0,21.0,NaN,NaN
1,2022-01-01T01:00:00+00:00,14.0,35.0,21.0,21.0,29.0,0.0037,20.0,0.01400,19.0,...,27.0,0.012,35.0,14.0,12.0,30.0,36.0,36.0,NaN,NaN
2,2022-01-01T02:00:00+00:00,25.0,44.0,22.0,29.0,35.0,0.0034,23.0,0.01300,29.0,...,41.0,0.012,34.0,18.0,16.0,28.0,46.0,32.0,NaN,NaN
3,2022-01-01T03:00:00+00:00,27.0,56.0,20.0,23.0,32.0,0.0044,27.0,0.01100,28.0,...,37.0,0.011,39.0,36.0,20.0,32.0,48.0,54.0,NaN,NaN
4,2022-01-01T04:00:00+00:00,26.0,49.0,20.0,27.0,38.0,0.0041,27.0,0.01200,30.0,...,44.0,0.011,37.0,33.0,29.0,33.0,40.0,84.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16671,2024-06-30T19:00:00+00:00,12.0,13.0,9.7,12.0,NaN,0.0290,7.0,0.00045,NaN,...,NaN,NaN,32.0,13.0,12.0,18.0,28.0,NaN,NaN,15.0
16672,2024-06-30T20:00:00+00:00,2.5,11.0,9.7,7.0,10.0,0.0320,9.0,NaN,11.0,...,NaN,NaN,17.0,12.0,NaN,20.0,32.0,13.0,NaN,2.5
16673,2024-06-30T21:00:00+00:00,17.0,13.0,11.0,6.0,13.0,0.0270,11.0,0.00210,NaN,...,NaN,NaN,29.0,10.0,NaN,4.0,12.0,NaN,NaN,11.0
16674,2024-06-30T22:00:00+00:00,NaN,NaN,0.5,7.0,10.0,0.0240,10.0,0.00590,9.7,...,NaN,NaN,NaN,6.0,1.2,9.0,17.0,NaN,NaN,NaN


In [3]:
sensor_ids = [20466, 34845, 34841, 35394, 35577, 35843, 36047, 36066, 36064, 36092, 35606]
sensor_colnames = ["datetime"] + [f"sid_{sensor_id}" for sensor_id in sensor_ids]

sensor_measurements = pivot_df[sensor_colnames]

Next, we define the SQL statement to create the table, establish a connection to the database, and upload our data. Finally, we perform a simple SELECT query to verify that the data has been successfully uploaded.

In [5]:
sensor_measurements.describe()

,sid_20466,sid_34845,sid_34841,sid_35394,sid_35577,sid_35843,sid_36047,sid_36066,sid_36064,sid_36092,sid_35606
count,14965.000000,15049.000000,15287.000000,16072.000000,15337.000000,15720.000000,15203.000000,15454.000000,14769.000000,14809.000000,15420.000000
mean,16.876124,0.016325,20.976581,0.014947,15.514742,44.825382,20.132415,46.472499,26.024342,20.370471,33.134760
std,9.870924,0.008371,11.963223,0.016251,9.123129,20.933911,11.957232,16.899530,11.942643,10.764739,17.882659
min,0.000000,0.000006,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000
25%,10.000000,0.010000,12.000000,0.002800,9.400000,29.000000,11.000000,34.000000,18.000000,13.000000,21.000000
50%,16.000000,0.015000,19.000000,0.008600,14.000000,42.000000,18.000000,45.000000,25.000000,19.000000,31.000000
75%,22.000000,0.021000,27.000000,0.022000,20.000000,58.000000,27.000000,57.000000,33.000000,26.000000,43.000000
max,230.000000,0.061000,110.000000,0.190000,140.000000,230.000000,140.000000,130.000000,92.000000,110.000000,400.000000


In [4]:
create_table_statement = """
DROP TABLE if exists raw_training_data;
CREATE TABLE raw_training_data (
    datetime TIMESTAMPTZ,
    sid_20466 FLOAT,
    sid_34845 FLOAT,
    sid_34841 FLOAT,
    sid_35394 FLOAT,
    sid_35577 FLOAT,
    sid_35843 FLOAT,
    sid_36047 FLOAT,
    sid_36066 FLOAT,
    sid_36064 FLOAT,
    sid_36092 FLOAT,
    sid_35606 FLOAT
);
"""

conn_params = {
    'host': 'localhost',
    'port': '5433',
    'dbname': 'aq_data',
    'user': 'postgres',
    'password': 'magedb_password',
}

# Connect to the database
with psycopg.connect(**conn_params) as conn:
    conn.autocommit = True
    cur = conn.cursor()
    cur.execute(create_table_statement)

    # Insert data into the table row by row
    insert_query = """
    INSERT INTO raw_training_data (datetime, sid_20466, sid_34845, sid_34841, sid_35394, sid_35577, sid_35843, sid_36047, sid_36066, sid_36064, sid_36092, sid_35606) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

    for i, row in sensor_measurements.iterrows():
        cur.execute(insert_query, (
            row['datetime'], row['sid_20466'], row['sid_34845'], row['sid_34841'], row['sid_35394'], row['sid_35577'],
            row['sid_35843'], row['sid_36047'], row['sid_36066'], row['sid_36064'], row['sid_36092'], row['sid_35606']
        ))
    
    cur.execute(f"SELECT COUNT(*) FROM raw_training_data")
    row_count = cur.fetchone()[0]
    print(f"Number of rows: {row_count}")

Number of rows: 16676
